# Context Engineering: A Hands-on Introduction

Welcome to this tutorial on **Context Engineering**! In the world of Large Language Models (LLMs), you've probably heard of Prompt Engineering. While prompt engineering is about crafting the perfect question, context engineering is about providing the LLM with the right *information* to answer that question. It's the art and science of designing systems that feed LLMs the relevant context they need to perform a task effectively. 

Think of it like this: if you ask a historian a question, they'll give you a much better answer if they have their library of books and research papers handy. Context engineering is about building that library for your LLM and making sure it can find the right book at the right time. 

## Why is Context Engineering Important?

LLMs have a limited "memory" called the **context window**. They can only consider a certain amount of information at a time. Context engineering helps us make the most of this limited space by:

* **Improving Accuracy:** By providing relevant, factual information, we can reduce the chances of the LLM "hallucinating" or making things up.
* **Enhancing Relevance:** We can steer the LLM's responses to be more specific and tailored to the user's needs.
* **Enabling Complex Tasks:** For tasks that require knowledge of specific documents or conversations, context engineering is essential. 

### Context-Problem Fit (CPF)
A core idea you'll rely on throughout this notebook is achieving strong **Context-Problem Fit (CPF)**. CPF means the context you assemble (retrieved documents, system instructions, tool schemas, memory, artifacts, etc.) is:

1. **Necessary** — each piece directly supports solving the current task.
2. **Sufficient** — taken together, the context eliminates major ambiguity and minimizes guesswork.
3. **Efficient** — it fits within token limits without bloating the prompt with redundant or low-signal content.

Great agents fail more often from poor CPF than from weak model capability. Always ask: *Does this context chunk increase the probability of a correct, useful answer for this specific problem?* If not, refine retrieval, compress, filter, or enrich.

This notebook will walk you through the basics of context engineering, from a simple Retrieval-Augmented Generation (RAG) system to System Prompts, Tool Schemas, and Message Buffer, and Files & Artifacts etc. Let's get started! 🚀

## 1. Setting Up Our Environment

First, let's get our environment ready. We'll be using the `dotenv` library to load our API keys from a `.env` file. This is a good practice for keeping your secrets safe and out of your code. 

You'll also need to install the necessary libraries. You can do this by running the following command in your terminal:

```bash
pip install python-dotenv langchain langchain-openai faiss-cpu
```

Now, create a `.env` file in the same directory as this notebook. Add your OpenAI API key to this file like so:

```
OPENAI_API_KEY="your_api_key_here"
```

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key from the environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is loaded
if openai_api_key:
    print("API Key loaded successfully!")
    # Initialize the main LLM we will use throughout the tutorial
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
else:
    print("API Key not found. Make sure you have a .env file with your OPENAI_API_KEY.")

API Key loaded successfully!


## 2. Context Components

### 2.1 The Knowledge Base: RAG with Vector Stores

As we first explored, a primary form of context is an external **knowledge base**. Retrieval-Augmented Generation (RAG) is the technique of retrieving relevant data from a knowledge base (like a vector database) and providing it to the LLM. This is a foundational concept in context engineering.

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Our knowledge source
knowledge_base = """
Planet Xylos has two moons, Zylo and Nylo. The native inhabitants are called the Xylotians, a species of intelligent, six-legged creatures. They are a peaceful and technologically advanced civilization.
"""

# Create a vector store retriever from the knowledge base
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_text(knowledge_base)
vectorstore = FAISS.from_texts(texts=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

# Create the RAG chain
rag_template = "Answer the question based only on the following context:\n{context}\n\nQuestion: {question}"
rag_prompt = ChatPromptTemplate.from_template(rag_template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("--- RAG Example ---")
print(rag_chain.invoke("How many moons does Xylos have?"))

--- RAG Example ---
Xylos has two moons.


Context is much more than just the RAG knowledge base. An advanced agent uses several types of context to understand its goals, capabilities, and history. Let's explore these.

### 2.2 System Prompt & System Metadata

The **System Prompt** defines the agent's persona, high-level instructions, and constraints. **System Metadata** provides the agent with information about its own state, like the current time or message history stats. We can combine these to create a more aware agent.

In [4]:
from langchain_core.messages import SystemMessage, HumanMessage
import datetime

# Simulate metadata
message_history = ["user: hello", "assistant: hi there!"]
metadata = {
    "current_time": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "message_count": len(message_history)
}

# Craft the system prompt with metadata included
system_prompt_string = f"""
You are a helpful assistant named 'Agent-7'. 
Your core directive is to be as concise as possible in your responses. 

--- System Metadata ---
Current Time: {metadata['current_time']}
Current Conversation Length: {metadata['message_count']} messages
--- End Metadata ---
"""

messages = [
    SystemMessage(content=system_prompt_string),
    HumanMessage(content="What time is it? What's the current conversation Length? What is the primary directive?"),
]

print("--- System Prompt Example ---")
response = llm.invoke(messages)
print(response.content)

--- System Prompt Example ---
- Current Time: 10:47:53
- Conversation Length: 2 messages
- Primary Directive: Be as concise as possible.


### 2.3 Tool Schemas

**Tool Schemas** are definitions of the functions or capabilities the agent can use. By providing the LLM with these schemas, it learns what actions it can perform to accomplish a task. This is a core component of making agents that can interact with the outside world.

In [5]:
from langchain_core.tools import tool

# Define a tool schema using a decorator
@tool
def get_current_stock_price(symbol: str) -> float:
    """Gets the current stock price for a given ticker symbol."""
    # In a real app, this would call a stock API.
    if symbol.upper() == "GOOGL":
        return 175.57
    elif symbol.upper() == "AAPL":
        return 214.29
    else:
        return 0.0

# Bind the tool to the LLM
llm_with_tools = llm.bind_tools([get_current_stock_price])

print("--- Tool Schema Example ---")
# The LLM's response will contain a tool_call object if it decides to use the tool
ai_msg = llm_with_tools.invoke("What is the stock price of Google?")
print("AI message contains a tool call:")
print(ai_msg.tool_calls)

--- Tool Schema Example ---
AI message contains a tool call:
[{'name': 'get_current_stock_price', 'args': {'symbol': 'GOOGL'}, 'id': 'call_6a9dl7ecUZy4w665Eg1osh6r', 'type': 'tool_call'}]


### 2.4 Message Buffer (Short-Term Memory)

The **Message Buffer** is the history of the conversation. It serves as the agent's short-term memory, allowing it to understand the flow of dialogue and refer back to previous points. `ConversationChain` in LangChain manages this automatically.

In [9]:
# Updated implementation replacing deprecated ConversationChain with RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

# In-memory store for session histories (for demo purposes)
session_store = {}

# Function LangChain will call to get (or create) the history object for a session
def get_session_history(session_id: str):
    return session_store.setdefault(session_id, ChatMessageHistory())

# Build a prompt that explicitly includes a placeholder for past messages
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a concise assistant that remembers the conversation."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Base chain (prompt -> llm)
base_chain = prompt | llm

# Wrap with message history support
chain_with_history = RunnableWithMessageHistory(
    base_chain,
    get_session_history=get_session_history,
    input_messages_key="input",      # key in invoke() input dict for new user message
    history_messages_key="history"    # name of the placeholder in the prompt
)

print("--- Message Buffer / Memory Example (RunnableWithMessageHistory) ---")

session_id = "demo-session"

# First user turn
response_1 = chain_with_history.invoke(
    {"input": "My name is Bob."},
    config={"configurable": {"session_id": session_id}}
)
# print("User: My name is Bob.")
# print(f"AI: {response_1}")

# Second user turn referring to prior context
response_2 = chain_with_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": session_id}}
)
# print("User: What is my name?")
# print(f"AI: {response_2}")

# Inspect the stored history messages
print("\n--- Memory Buffer Contents ---")
for m in session_store[session_id].messages:
    role = 'User' if isinstance(m, HumanMessage) else 'AI'
    print(f"{role}: {m.content}")

--- Message Buffer / Memory Example (RunnableWithMessageHistory) ---

--- Memory Buffer Contents ---
User: My name is Bob.
AI: Nice to meet you, Bob! How can I assist you today?
User: What is my name?
AI: Your name is Bob.


### 2.5 Files & Artifacts

This involves providing the agent with direct access to **Files & Artifacts** like PDFs, CSVs, or code files. This is similar to RAG but can be more direct, where the entire content of a file is loaded into the context to be summarized, analyzed, or transformed.

In [7]:
# First, let's create a dummy file to act as our artifact
file_content = """
PROJECT REQUIREMENTS DOCUMENT
1. The user interface must be blue.
2. The login button must be labeled 'Enter'.
3. The system must support up to 100 concurrent users.
"""
with open("requirements.txt", "w") as f:
    f.write(file_content)

# Now, load the file content
with open("requirements.txt", "r") as f:
    file_context = f.read()

file_prompt_template = """
You are a project manager assistant. Answer the user's question based on the following project requirements file.

--- File Content: requirements.txt ---
{file_context}
--- End File Content ---

Question: {user_question}
"""

file_prompt = ChatPromptTemplate.from_template(file_prompt_template)

file_chain = file_prompt | llm | StrOutputParser()

print("--- Files & Artifacts Example ---")
response = file_chain.invoke({
    "file_context": file_context,
    "user_question": "How many users must the system support?"
})
print(response)

# Clean up the dummy file
os.remove("requirements.txt")

--- Files & Artifacts Example ---
The system must support up to 100 concurrent users.


## Conclusion

Context engineering is a comprehensive discipline that involves strategically managing different types of information to build powerful and accurate AI agents. By moving beyond simple RAG, you can control an agent's persona, grant it new abilities with tools, give it a memory, and allow it to interact with files.

Mastering these techniques is the key to unlocking the full potential of Large Language Models. Keep experimenting! 🛠️